In [27]:
%pwd

'D:\\Work\\notebook\\metricstudy\\notebook'

In [28]:
BTC_ETH = "BTC_DASH"

poloniex_url_template = "https://poloniex.com/public?command=returnTradeHistory&currencyPair=%s&start=%s&end=%s"


In [29]:
#currency_pair: currency pair expression which is supported in poloniex, to: unixtime value
def get_url(currency_pair, end_unix):
    SIXMONTHS = 60*60*24*30*6
    ONEDAY = 60*60*24
    ONWWEEK = 7*ONEDAY
    
    date_range = ONEDAY
    start_unix = end_unix - date_range
    url = poloniex_url_template%(currency_pair, start_unix, end_unix)
    print (url)
    return url

In [30]:
import time
import datetime

def now():
    #now = int(time.gmtime())
    now = datetime.datetime.utcnow()
    now = int(time.mktime(now.timetuple()))
    #print(type(now))
    print (now)
    return now
#1493047494
now()

1493372639


1493372639

In [31]:
import datetime

def get_gmttime(lastdate):
    #print(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S"))
    endtime = int(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S").timestamp())+(9*60*60)
    #print(endtime)
    return endtime


In [32]:
import requests

def get_bth_eth(lasttime):
    url = get_url(BTC_ETH, lasttime)
    r = requests.get(url)
    js = r.json()
    
    lastdate = js[-1]['date']
    endtime = get_gmttime(lastdate)

    print ("downloaded %d items, by %s"%(len(js), lastdate))
    
    return (js, endtime)

#print(js[:2])
#print(js[-2:])

In [33]:
def get_batch_list(total_size, batch_size=100):
    total = [a for a in range(0, total_size)]
    #print (total)
    
    res = list()
    while len(total) > 0:
        bs = min(batch_size, len(total))
        #print (bs)
        a = total[:bs]
        res.append(a)
        if bs < len(total):
            b = total[bs:]
            total = b
        else:
            total = []
            break
    return res
    

In [34]:
import MySQLdb

In [35]:
conn = MySQLdb.connect(db='poloniex', user='polo', passwd='polo')
curr = conn.cursor()

In [36]:
def execute_query(query):
    try:
        curr.execute(query)
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print ("MySQL Error [%d]: %s" % (e.args[0], e.args[1]))
    except IndexError:
        print ("MySQL Error: %s" % str(e))

def get_db_size():
    curr.execute("SELECT COUNT(*) FROM btc_eth")
    (number_of_rows,)=curr.fetchone()
    return number_of_rows

In [37]:
#make insert statement
def add_data(trade_json):
    batch_size = 1000
    batch = get_batch_list(len(trade_json), batch_size)

    for bs in batch:
        query = "insert into btc_dash values "

        first = True
        for b in bs:
            item = trade_json[b]
            #print (item)

            value = "(%d,%d,%s,%s,%s,'%s','%s')"%(item['globalTradeID'], item['tradeID'], item['total'], item['rate'], item['amount'], item['type'], item['date'])

            if first:
                first = False
            else:
                query += ','

            query += value
        #print ("processing %d"%(len(bs)))
        #print (query)
        
        execute_query(query)
        #print ("done@@")
    
    #print ("begin commit")
    if len(batch) > 0:
        conn.commit()
    #print ("end commit")
    
    print ("trade data count = %d"%(get_db_size()))
    

In [38]:
import time
import datetime

starttime = "2016-06-22 07:31:22"
starttime = "2015-08-08 05:11:37"
start_timestamp = int(time.mktime(datetime.datetime.strptime(starttime, "%Y-%m-%d %H:%M:%S").timetuple()))
print (start_timestamp)
    

1438978297


In [ ]:
lasttime = now()
#lasttime = start_timestamp
prev_lasttime = 0
while prev_lasttime != lasttime:
    prev_lasttime = lasttime
    (js_result, lasttime) = get_bth_eth(lasttime)
    print (lasttime)
    #print js_result
    #continue
        
    if len(js_result) == 0:
        break
    
    #add trade data to db
    add_data(js_result)
    

1493372660
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_DASH&start=1493286260&end=1493372660
downloaded 32168 items, by 2017-04-27 09:44:34
1493286274
trade data count = 21866853
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_DASH&start=1493199874&end=1493286274
downloaded 14445 items, by 2017-04-26 09:45:14
1493199914
MySQL Error [1062]: Duplicate entry '112984103' for key 'PRIMARY'
trade data count = 21866853
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_DASH&start=1493113514&end=1493199914
downloaded 12324 items, by 2017-04-25 09:45:24
1493113524
MySQL Error [1062]: Duplicate entry '112298114' for key 'PRIMARY'
trade data count = 21866853
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_DASH&start=1493027124&end=1493113524
downloaded 11190 items, by 2017-04-24 09:45:26
1493027126
MySQL Error [1062]: Duplicate entry '111774510' for key 'PRIMARY'
trade data count = 21866853
https://pol

In [35]:
len(js_result) 

30452